In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

True

## test the azure function using http

In [3]:
import requests

def read_data_from_file(filename):
    piezo_data = []
    mic_data = []
    with open(filename, 'r') as file:
        for line in file:
            # 去掉每行的首尾空白字符（如换行符）并分割数据
            try: 
                parts = line.strip().split(',')
                pizo_temp_data = int(parts[0])
                mic_temp_data = int(parts[1])
                if len(parts) == 2:  # 确保每行有两个数据点
                    piezo_data.append(pizo_temp_data)
                    mic_data.append(mic_temp_data)
                else:
                    raise Exception("Invalid data line")
            except Exception as e:
                # print(e)
                # if this line is not a valid data line, repeat the previous line (to keep the dimensions the same)
                piezo_data.append(piezo_data[-1])
                mic_data.append(mic_data[-1])
    print(len(piezo_data), len(mic_data))
    return piezo_data, mic_data

def send_data_to_function(url, piezo_data, mic_data):
    data = {
        "piezo": piezo_data,
        "mic": mic_data
    }
    # print(len(piezo_data), len(mic_data))
    # 发送POST请求
    response = requests.post(url, json=data, verify=False)

    # 处理响应
    if response.status_code == 200:
        # get json data
        print(response.json())
        # save to file
        with open('data/result.json', 'w') as file:
            file.write(response.text)
        return response.json()
    else:
        print("请求失败，状态码：", response.status_code)
        print("错误信息：", response.text)


In [ ]:
# 读取文件并准备数据
piezo, mic = read_data_from_file('data/sampleData.txt')
# 发送数据到本地Azure Function
result = send_data_to_function("https://omnisurface.azurewebsites.net/api/esp_rawdata_process", piezo, mic)

## Environmental Variables
- `Flag` (whether it is recording):
    - True: recording state, save the data to Azure storage
        - The blob file has the name of the current time (yy-mm-dd-hh-mm-ss).
    - False: predict state, send the data to Azure ML model
- `current_label`: The label of the current data. This will also be the filename of the incoming data.
- `data_count`: How many new data has been stored for the current label.

In [5]:
from azure.data.tables import TableServiceClient

# 连接到Azure Table Storage
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")


# 创建 TableServiceClient 实例
table_service = TableServiceClient.from_connection_string(conn_str=connection_string)

# 创建表，如果不存在的话
table_name = "envVariables"
table_service.create_table_if_not_exists(table_name=table_name)

### Update current environmental variables

In [6]:
def update_env_variables(record_state, label, data_count):
    connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
    table_service_client = TableServiceClient.from_connection_string(connection_string)
    table_client = table_service_client.get_table_client("envVariables")
    entity = {
        "PartitionKey": "enviroment",
        "RowKey": "variables",
        "Flag": record_state,
        "current_label": label,
        "data_count": data_count
    }
    table_client.upsert_entity(entity)    

### Get current environmental variables

In [7]:
def get_env_variables():
    connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
    table_service_client = TableServiceClient.from_connection_string(connection_string)
    table_client = table_service_client.get_table_client("envVariables")
    try:
        entity = table_client.get_entity(partition_key="enviroment", row_key="variables")
        return entity['Flag'], entity.get('current_label', 'default_label'), entity.get('data_count', 0)
    except Exception as e:
        print("Error reading state:", e)
        return None, None  # return None if there is an error


## Simulate backend refresh the label counter
after recording a new data, the counter will +1.

In [72]:
get_env_variables()
# store the state
record_state, label, data_count = get_env_variables()
update_env_variables(True, label, data_count + 1)  # set state

## Save data to Azure Storage Blob

- Outdated. This storage function has been integrated with the Azure Function.

In [8]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json

def upload_data_to_blob(data, label, data_name, connection_string, container_name):
    # 初始化 Blob 服务客户端
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    
    # 生成 Blob 名称
    blob_name = f"{data_name}.json"
    blob_client = container_client.get_blob_client(blob_name)
    
    # 检查 Blob 是否存在
    try:
        blob_client.get_blob_properties()
        # 如果 Blob 存在，下载现有内容，追加数据，然后重新上传
        stream = blob_client.download_blob()
        existing_data = json.loads(stream.readall())
        existing_data.append({"features": data, "label": label})
        new_data = json.dumps(existing_data)
    except Exception as e:
        # 如果 Blob 不存在，创建一个新的数据列表
        new_data = json.dumps([{"features": data, "label": label}])

    # 上传或更新 Blob
    blob_client.upload_blob(new_data, overwrite=True)


In [9]:
get_env_variables()

(True, 'Test', 18)

In [41]:
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
container_name = "omnisurface-ml-data"
data_name = "2024-05-03-00-32-53"
label = "single_tap"
data = result
upload_data_to_blob(data, label, data_name, connection_string, container_name)

NameError: name 'upload_data_to_blob' is not defined

## 🌟 Simulate entire workflow: Web app, ESP, Azure Functions

### 1️⃣ simulate web app click "new data collection" (toppest button)

In [13]:
# check if table exists
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
table_service_client = TableServiceClient.from_connection_string(connection_string)
table_service.create_table_if_not_exists(table_name="envVariables")

# reset the environmental variables
# 这时候还不需要把flag设置为true，因为还没有开始采集数据
update_env_variables(False, "", 0) # set initial label to blank and data count to 0

### 2️⃣ simulate data collection process: Web app & ESP

In [14]:
import os,glob
gestures = ["fist", "single-finger", "tap"]

# 🌟🌟🌟 模拟web app中选好gesture后点击"record"按钮，每次循环相当于用户选好一个手势，然后按下record按钮。循环体里就是按下record后需要干的事情
for gesture in gestures:
        update_env_variables(True, gesture, 0) # 开始采集这个手势的数据
        
        # 现在用本地读取文件的方式来模拟ESP端采集数据
        for file in glob.glob(f"data/combined_new/{gesture}/*.txt"):
                piezo, mic = read_data_from_file(file)
                send_data_to_function("https://omnisurface.azurewebsites.net/api/esp_rawdata_process", piezo, mic)
        
        # 🛑🛑🛑🛑 这里需要增加前端读取env_variables中的data_count，然后在前端显示成功收集了多少手势；比如如果成功收集10个，然后弹出“数据收集成功”！

# 结束所有上传，重置环境变量
update_env_variables(False, "", 0) # set initial label to blank and data count to 0
        

3000 3000


/Users/pengyuecheng/Documents/UW/TECHIN 515/final/Azure function/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'omnisurface.azurewebsites.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'features': [6234199.53857287, 249487.205971295, 165900.98895036447, 45752.17889940424, 39170.631282525224, 57158.467807524394, 75570.31972237409, 50645.28579135343, 64771.756296077096, 43064.76101460497, 9704.546839171298, 25325.10670879511, 57723.75526809031, 76112.5470952417, 87076.78822392141, 111144.76541306045, 117357.49465561697, 80117.52771800378, 49701.55805259213, 3415.0006647493237, 21753.77523707715, 60082.94371065116, 59740.337711197615, 52101.086450547504, 87083.03664348561, 82772.60140314649, 14507.067570437108, 103692.66972674984, 156293.64452575098, 177545.87727085603, 144326.97899350442, 88612.23509677158, 125829.65555125457, 247902.3240596045, 240219.0524758183, 181712.462466874, 191004.1226037596, 463703.8302874777, 693456.201374605, 796294.3321758771, 750819.5819509324, 577851.5600853337, 316710.7876577456, 101848.94726084173, 92226.52166256869, 171264.28487285617, 162640.40366973792, 80468.26018479418, 18326.63331482019, 48739.1864046278, 69969.54449376841, 45899

/Users/pengyuecheng/Documents/UW/TECHIN 515/final/Azure function/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'omnisurface.azurewebsites.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'features': [6071743.538572868, 182504.80439886003, 158470.94782502984, 102896.0325631707, 137676.94683412416, 90025.6543099192, 74060.94404055478, 25903.32227341911, 21212.263436999645, 47152.108146557606, 105551.5054261363, 105832.766991855, 57493.81327353826, 69810.42446835837, 59718.169751620175, 69289.77223427568, 76754.91704919293, 22152.633797362494, 83794.44102066272, 87018.81362102237, 74534.33384324465, 38970.3062726457, 16092.948313144168, 41115.28345375389, 90725.01269036814, 129173.53196514993, 154377.61615623467, 166528.43916135532, 131759.54492294395, 33000.21132148708, 72533.24189792454, 76506.69675642022, 74853.76947227879, 172643.36772358484, 250225.88373308792, 366824.2501450159, 401050.9636320163, 486201.76483280107, 477853.9215417779, 448560.360515915, 379156.0854278628, 351832.9671883973, 310813.53716408537, 290094.69267787633, 205212.9130676586, 116401.71073604713, 71830.7586092177, 44900.78369357861, 54915.95485238153, 119179.7447768826, 79541.294195935, 21385.

...

{'features': [6303133.316959799, 118587.89893760778, 111225.75534200367, 54841.776199907115, 76403.64251216307, 105699.22740157753, 169963.96014127677, 51245.12600259511, 96961.74206007486, 63984.90073103009, 44800.66327062444, 46893.593952585514, 30853.782028766396, 38521.407100506476, 57494.67147792338, 74138.4706965679, 46577.69327897177, 49599.03396488991, 33637.22219416052, 5622.165847113355, 15859.649306513264, 39534.85073073239, 46194.265036606186, 45110.40364323957, 34921.84503096641, 41288.42094880517, 75217.15017231088, 88931.83191315198, 93757.4842112594, 91165.2776212298, 81953.15114257969, 38461.124555923496, 71760.75907345625, 131151.4516838024, 147894.19700010537, 322622.9101149922, 400944.0085259795, 623151.9368344271, 688919.4619238797, 702804.6924272949, 623060.8178781009, 401474.0004125365, 185082.54915220613, 108350.0903807252, 81833.69422336234, 224249.97144834683, 293122.53076049505, 286961.2728949563, 224250.13310863933, 149375.49700449844, 133169.28461170816, 97

/Users/pengyuecheng/Documents/UW/TECHIN 515/final/Azure function/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'omnisurface.azurewebsites.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'features': [6156675.53857287, 89071.75597473452, 41637.25783921617, 83180.11828299532, 75379.16154955774, 73036.40306395726, 95605.0699241917, 26417.52341998465, 28119.39331117541, 14289.030276252977, 35254.88825293288, 35285.600884151754, 15682.771112965145, 29396.162692434278, 6805.490990388919, 28422.963692165533, 104293.80427568399, 73902.15181123173, 59288.31027348413, 34738.99927065973, 6687.033585717568, 33419.443313197095, 35990.35910262094, 45252.518620951705, 53185.45926326267, 65270.22261866008, 42335.442725206245, 74387.34186047887, 80770.45274994771, 111935.10503107807, 185136.665353621, 266820.8920262749, 267489.20364928036, 263138.52811609616, 265498.12966793607, 215543.8052495916, 175026.2249317008, 147930.29535507204, 122578.53451586765, 172625.24414921386, 88803.63690737002, 107172.87733161, 194193.68760963227, 271121.65484780405, 273398.3505089456, 212552.8603054574, 233756.3846943312, 184817.77926854743, 149283.25134061856, 145571.05668476367, 148656.55281593287, 

/Users/pengyuecheng/Documents/UW/TECHIN 515/final/Azure function/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'omnisurface.azurewebsites.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'features': [6330279.249638191, 164470.5983778455, 109078.11173146438, 20227.34226338184, 61742.06709768094, 20128.959989200426, 127315.0365808906, 39586.95926682691, 69830.76419918436, 48548.75667955425, 48753.76992432096, 27947.499509815738, 39469.15934399401, 71923.02408775268, 70393.47100351498, 55217.52212707888, 71866.12608191956, 61171.13990252111, 51158.990614400354, 40027.02185576653, 67910.84172679193, 55845.60388080867, 55067.31758954029, 42600.147259019184, 41465.866186844, 54979.14779835857, 50077.41127493735, 51265.14091808034, 82156.42467367799, 85768.85452146943, 97358.46769079291, 97443.65817241532, 202947.47748484655, 242597.3668924841, 136202.6181855214, 145690.9831911898, 293632.2921504487, 650657.9312735413, 842065.0864017367, 892401.56933481, 730236.6435309409, 498593.638675661, 139153.8712672841, 31564.797025807366, 232959.0442734561, 262869.30317254, 243440.77833652488, 155227.9074123372, 113705.16886485662, 165070.61232607055, 236696.8382195113, 215365.7649471

### 3️⃣ simulate web app: click "train model" button

In [16]:
# train model
import requests

def call_train_model_function():
    url = "http://omnisurface.azurewebsites.net/api/train_model"
    response = requests.post(url)  # 使用POST请求，因为你可能会处理数据
    
    if response.status_code == 200:
        print("成功: ", response.text)
    else:
        print("失败: ", response.status_code, response.text)

call_train_model_function()
# 🌟 在前端可以反馈模型 accuracy，调用这个函数call_train_model_function会在返回中包括accuracy

成功:  Model and scaler uploaded successfully. Model trained successfully with 24 training samples and 6. Test accuracy: 1.00


## Use the trained model

First turn the "recording" flag to False. Here we are doing it one more time to ensure the flag is False.

In [11]:
get_env_variables()
# store the state
record_state, label, data_count = get_env_variables()
update_env_variables(False, label, data_count)  # set state

Then just call the upload function (simulate the process of sending ESP raw data to the Azure Function): `esp_rawdata_process`

In [4]:
import requests

def send_data_to_function(url, piezo_data, mic_data):
    data = {
        "piezo": piezo_data,
        "mic": mic_data
    }
    print(data)
    # 发送POST请求
    response = requests.post(url, json=data)

    # 处理响应
    if response.status_code == 200:
        # get json data
        print(response.json())
        # save to file
        with open('data/result.json', 'w') as file:
            file.write(response.text)
        return response.json()
    else:
        print("请求失败，状态码：", response.status_code)
        print("错误信息：", response.text)

# 读取文件并准备数据
piezo, mic = read_data_from_file('data/combined_new/fist/output_20240505-214508.txt')
# 发送数据到本地Azure Function
result = send_data_to_function("https://omnisurface.azurewebsites.net/api/esp_rawdata_process", piezo, mic)

3000 3000
{'piezo': [2067, 2076, 2078, 2073, 2072, 2073, 2075, 2075, 2077, 2076, 2073, 2065, 2063, 2043, 2040, 2032, 2058, 2053, 2063, 2072, 2089, 2105, 2120, 2147, 2158, 2169, 2201, 2224, 2259, 2266, 2289, 2313, 2335, 2361, 2382, 2401, 2429, 2442, 2453, 2464, 2461, 2468, 2463, 2470, 2460, 2463, 2473, 2463, 2470, 2455, 2440, 2429, 2419, 2405, 2403, 2399, 2410, 2416, 2425, 2471, 2511, 2571, 2628, 2697, 2773, 2852, 2941, 3032, 3131, 3213, 3290, 3369, 3415, 3461, 3488, 3493, 3485, 3448, 3401, 3326, 3239, 3136, 3032, 2905, 2779, 2641, 2499, 2357, 2211, 2071, 1910, 1763, 1605, 1451, 1272, 1121, 971, 827, 685, 544, 427, 300, 197, 110, 40, 2, 0, 0, 0, 9, 59, 145, 229, 353, 469, 600, 756, 902, 1059, 1235, 1384, 1545, 1691, 1837, 1971, 2103, 2216, 2328, 2455, 2559, 2663, 2769, 2876, 2976, 3105, 3221, 3352, 3483, 3623, 3764, 3913, 4053, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4095, 4060, 3925,

In [5]:
result

{'prediction': 'Rub',
 'label_proba_dict': {'Fist': 0.32, 'Rub': 0.46, 'Single Finger Tap': 0.22}}

## Outdated code

In [ ]:
def deploy_model(model_name, model_version):
    # 加载工作区
    ws = Workspace.from_config()

    # 获取模型
    model = Model(ws, name=model_name, version=model_version)

    # 设置环境
    env = Environment('deploy-env')
    cd = CondaDependencies.create(pip_packages=['numpy', 'scikit-learn', 'azureml-defaults'])
    env.python.conda_dependencies = cd

    # 推理配置🛑🛑🛑🛑🛑
    inference_config = InferenceConfig(entry_script='score.py', environment=env)

    # 部署配置
    deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

    # 部署模型
    service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config)
    service.wait_for_deployment(show_output=True)
    print(service.state)
    print("Scoring URI: " + service.scoring_uri)

def register_model(model, model_name, description="My random forest model"):
    # Connect to Azure ML Workspace
    ws = Workspace.from_config()  # Assumes you have a config.json file in your Azure Function or set environment variables

    # Save the model to a temporary directory
    os.makedirs('outputs', exist_ok=True)
    model_path = os.path.join('outputs', 'model.pkl')
    joblib.dump(model, model_path)

    # Register the model
    registered_model = Model.register(workspace=ws,
                                      model_path=model_path,
                                      model_name=model_name,
                                      description=description)
    print(f"Model registered: {registered_model.name}, ID: {registered_model.id}, Version: {registered_model.version}")
    return registered_model


In [36]:
import numpy as np
max_proba = np.max(np.array([[1,2,4,5,67,8],[1,2,4,5,111,8]]))


In [37]:
max_proba

111